# Import relevant libraries

In [1]:
import os
import glob
import re
import pandas as pd

# Setup path

In [2]:
# set up paths
base_path = '../../datasets'
raw_data_path = os.path.join(base_path, 'raw_data')
cleaned_data_path = os.path.join(base_path, 'cleaned_data')

# change directory to raw_data_path
os.chdir(raw_data_path)

# Load data

In [3]:
# get folder names
folders = [name for name in os.listdir() if os.path.isdir(name)]
print(folders)

['1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']


In [4]:
# initiate a table
denorm_table = []

# load and merge all files from 1992-2020
for folder in folders:
    
    # get all file paths a folder
    file_paths = glob.glob(os.path.join(folder, '*.csv'))
    
    for file_path in file_paths: 
        
        # read file
        file = pd.read_csv(file_path)
        
        # drop unmatched column
        if 'transfer_period' in file.columns:
            file.drop(columns = 'transfer_period', inplace = True)
        
        # concat table
        if len(denorm_table) == 0:
            denorm_table = file
        else:
            # check if columns of the two tables are the same 
            assert all(denorm_table.columns == file.columns), 'Columns do not match!'
            
            # concat table
            denorm_table = pd.concat([denorm_table, file])

In [5]:
denorm_table.reset_index(drop = True, inplace = True)
denorm_table.head()

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,fee_cleaned,league_name,year,season
0,Fortuna Sittard,René Hofman,31.0,Right Winger,Roda JC,?,in,NaN,Eredivisie,1992,1992/1993
1,Fortuna Sittard,Peter Schmitz,21.0,Forward,Helmond Sport,?,in,NaN,Eredivisie,1992,1992/1993
2,Fortuna Sittard,Kenneth Nysaether,22.0,Centre-Forward,Lillestrøm SK,?,in,NaN,Eredivisie,1992,1992/1993
3,Fortuna Sittard,Arno van Zwam,22.0,Goalkeeper,Fortuna U19,-,in,0.0,Eredivisie,1992,1992/1993
4,Fortuna Sittard,Jerry Taihuttu,22.0,Centre-Forward,Helmond Sport,Loan,out,0.0,Eredivisie,1992,1992/1993


# Preprocessing

- Remove duplicated rows

In [6]:
denorm_table.drop_duplicates(inplace = True)

**Clean data**  
**Issues:**  
- `club_name` and `club_involved_name` columns are in a different format *(len(`club_involved_name`.unique()) >>> len(`club_name`.unique))*  
- Row counts for `transfer_movement` column 'in' > 'out'

**Solution:**
- Only keep relevant data points: each transaction should have `transfer_movement` = 'in' and 'out' (ledger format)  
- Use the club name format in `club_involved_name` column - more specific 

In [7]:
group = ['player_name', 'age', 'position', 'fee', 'fee_cleaned', 'year', 'season']
table = denorm_table.groupby(group).agg({'transfer_movement': 'unique'})

In [8]:
# get length of transfer_movement lists
table['len'] = [len(x) for x in table['transfer_movement']]

In [9]:
# len == 2 means there are ['in', 'out']
table = table[table['len'] == 2]

In [10]:
table.reset_index(inplace = True)

In [11]:
joined_table = pd.merge(table, denorm_table, on = group, how = 'left')

In [12]:
# remove irrelevant columns
joined_table.drop(columns = ['transfer_movement_x', 'len'], inplace = True)
# rename columns
joined_table.rename(columns = {'transfer_movement_y': 'transfer_movement'}, inplace = True)

In [13]:
print(f'number of observation: {len(joined_table)}')
joined_table.head()

number of observation: 55158


,player_name,age,position,fee,fee_cleaned,year,season,club_name,club_involved_name,transfer_movement,league_name
0,Aapo Halme,21.0,Centre-Back,£203Th.,0.203,2019,2019/2020,Leeds United,Barnsley FC,out,Championship
1,Aapo Halme,21.0,Centre-Back,£203Th.,0.203,2019,2019/2020,Barnsley FC,Leeds,in,Championship
2,Aaron Cresswell,24.0,Left-Back,£4.28m,4.280,2014,2014/2015,Ipswich Town,West Ham,out,Championship
3,Aaron Cresswell,24.0,Left-Back,£4.28m,4.280,2014,2014/2015,West Ham United,Ipswich,in,Premier League
4,Aaron Hughes,25.0,Centre-Back,£1.35m,1.350,2005,2005/2006,Aston Villa,Newcastle,in,Premier League


In [14]:
joined_table.to_csv(os.path.join(cleaned_data_path, 'node_edge_data.csv'), index = False)

- Get list of players who were bought at least once (for performance evaluation)

In [15]:
pattern = r"£"

slicer = []

for i in range(len(joined_table)):
    val = bool(re.match(pattern, str(joined_table['fee'][i])))
    slicer.append(val)

perf_eval_table = joined_table[slicer]
perf_eval_table.reset_index(drop = True, inplace = True)
print(f'number of observation: {len(perf_eval_table)}')
perf_eval_table.head()

number of observation: 16102


,player_name,age,position,fee,fee_cleaned,year,season,club_name,club_involved_name,transfer_movement,league_name
0,Aapo Halme,21.0,Centre-Back,£203Th.,0.203,2019,2019/2020,Leeds United,Barnsley FC,out,Championship
1,Aapo Halme,21.0,Centre-Back,£203Th.,0.203,2019,2019/2020,Barnsley FC,Leeds,in,Championship
2,Aaron Cresswell,24.0,Left-Back,£4.28m,4.280,2014,2014/2015,Ipswich Town,West Ham,out,Championship
3,Aaron Cresswell,24.0,Left-Back,£4.28m,4.280,2014,2014/2015,West Ham United,Ipswich,in,Premier League
4,Aaron Hughes,25.0,Centre-Back,£1.35m,1.350,2005,2005/2006,Aston Villa,Newcastle,in,Premier League


In [16]:
perf_eval_table.to_csv(os.path.join(cleaned_data_path, 'perf_eval_table.csv'), index = False)